In [115]:

# %pip install selenium

In [46]:
# from datetime import datetime
# from datetime import timedelta
# from dateutil import relativedelta

# year_list = [i for i in range(2005, 2025)]
# month_list = [i for i in range(1, 13)]

# start_date_list = []
# end_date_list = []

# def getMonthRage(year, month):
#     this_month = datetime(year=year, month=month, day=1).date()
#     next_month = this_month + relativedelta.relativedelta(months=1)

#     first_day = this_month
#     last_day = next_month - timedelta(days=1)
    
#     start_date_list.append(str(first_day))
#     end_date_list.append(str(last_day))

# for year in year_list:
#     for month in month_list:
#         getMonthRage(year, month)


In [73]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
announce_dates = pd.read_csv("C:/Users/SesacPython/Downloads/기준금리날짜.csv")

In [77]:
full_announce_dates = announce_dates.iloc[:206]
announce_dates = full_announce_dates.iloc[:20]

In [47]:
date_lists = []
news_lists = []
final_news_links = []

for announce_date, start_date, end_date in zip(announce_dates.announce_date, announce_dates.start, announce_dates.end):

    testurl = f"https://www.mk.co.kr/search/news?word=%EA%B8%88%EB%A6%AC&dateType=direct&startDate={start_date}&endDate={end_date}"
    driver = webdriver.Chrome()
    driver.get(testurl)

    try:
        # '더보기' 버튼이 발견되고 클릭 가능한 동안 계속 반복
        while True:
            try:
                # '더보기' 버튼이 클릭 가능할 때까지 대기
                wait = WebDriverWait(driver, 10)
                more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'btn.basic.w_f.drop_sub_news_btn.h_s.ic_before')))
                
                # '더보기' 버튼 클릭
                driver.execute_script("arguments[0].click();", more_button)
                
                # 서버에 과부하를 주지 않도록 작은 지연을 추가 (또는 애니메이션 완료를 위해)
                time.sleep(2)
            
            except TimeoutException:
                # 버튼이 지정된 시간 내에 발견되지 않으면 반복문 종료
                print("더 이상 클릭 가능한 '더보기' 버튼이 없습니다.")
                break
            
    except Exception as e:
        print(f"오류 발생: {e}")

    # =====================================================================

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    news_items =soup.select("a.news_item")
    news_links = []

    for i in news_items:
        href = i.attrs['href']
        news_links.append(href)

    # 각 링크별 본문 내용 수집

    for news_link in news_links:
        # 기사 내용
        res = requests.get(news_link)
        soup = BeautifulSoup(res.text, 'html.parser')
        news_contents = soup.select(".news_cnt_detail_wrap")
        
        # 구독자 전용 기사 pass
        if news_contents == []:
            continue

        # 기사 날짜
        res2 = requests.get(news_link)
        soup = BeautifulSoup(res2.text, 'html.parser')
        news_date = soup.select("dl.registration > dd")
        
        
        news_text = news_contents[0].get_text()
        news_date = news_date[0].get_text()[:10]
        news_lists.append(news_text)
        final_news_links.append(news_link)
        
        date_lists.append(announce_date)
        # date_lists.append(news_date)

    # news_dict[announce_date] = [[], []]
    # news_dict[announce_date][0] = news_lists
    # news_dict[announce_date][1] = final_news_links


더 이상 클릭 가능한 '더보기' 버튼이 없습니다.
더 이상 클릭 가능한 '더보기' 버튼이 없습니다.


In [60]:
df = pd.DataFrame({'news_lists':news_lists, 'links':final_news_links}, index=date_lists)

In [61]:
df

,news_lists,links
2005-02-15,\n내전으로 폐허가 된 레바논의 재건사업을\n진두지휘했던 라피크 하리리(60) ...,https://www.mk.co.kr/news/all/3500326
2005-02-15,\n ========================================\...,https://www.mk.co.kr/news/all/3500240
2005-02-15,\n 다음달 5.5대 1 감자를 앞두고 있는 LG카드가 급등했다.\n\n 1...,https://www.mk.co.kr/news/all/3500183
2005-02-15,"\n ""지수 1000선 넘을 수 있을까.""\n\n 종합주가지수가 5년래 최고...",https://www.mk.co.kr/news/all/3500184
2005-02-15,\n ◆달아오른 증시 960 훌쩍◆\n\n 14일 주가가 급등한 배경은 무엇...,https://www.mk.co.kr/news/all/3500154
...,...,...
2005-03-10,\n올들어 채권시장의 투자심리가 취약해진 가운데\n금리의 등락폭이 크게 늘어났다....,https://www.mk.co.kr/news/all/3503184
2005-03-10,\n주식시장에서 개미들의 직접 투자와 간접 투자 모\n두 활기를 띠고 있다.\n ...,https://www.mk.co.kr/news/all/3503181
2005-03-10,"\n <그린스펀 재정적자, 채권시장 발언 추가>\n 김대영ㆍ이래운 특파원 = ...",https://www.mk.co.kr/news/all/3503111
2005-03-10,"\n앨런 그린스펀 미 연방준비제도이사회(FRB)\n의장은 16일(현지시간) ""실...",https://www.mk.co.kr/news/all/3503101
